### Analyzing the language in news articles

How are accidents involving pedestrians and cyclists described?

In [15]:
import pandas as pd
import spacy

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
nlp = spacy.load('en_core_web_lg')

In [5]:
def is_passive(sent) -> bool:
    '''
    TODO this is just something I put together in a very adhoc way and
    is probably not great, although sort of seems to do OK. Should improve.
    '''
    doc = nlp(sent)
    
    tags = [token.dep_ for token in doc]
    
    contains_agent = 'agent' in tags
    contains_nsubjpass = 'nsubjpass' in tags
    contains_aux_verb = 'auxpass' in tags 
    
    return contains_agent or contains_nsubjpass or contains_aux_verb

In [9]:
is_passive("A pedestrian was struck")

True

In [10]:
is_passive("A car struck a pedestrian")

False

This data was scraped from https://www.newsbreak.com/. @TODO George can fill in details.

In [11]:
all_df = pd.read_csv("articles-all.csv")
all_df.head()

,Unnamed: 0,Date,City,State,Title,Short Description,Long Description,Involves cyclist or pedestrian,URL
0,0,2022-01-12,Cleves,OH,2 injured in wrong-way crash on I-275 in Kento...,"KENTON COUNTY, Ky. — Two people were injured i...","KENTON COUNTY, Ky. — Two people were injured i...",False,https://www.newsbreak.com/news/2483276062044/2...
1,1,2022-01-11,Cleves,OH,"Pedestrian hit, killed in West Chester on Satu...",A spokesperson for West Chester Township said ...,"Read the latest Cincinnati, Ohio news and weat...",True,https://www.newsbreak.com/news/2483036184238/p...
2,2,2022-01-11,Cleves,OH,Officials working to determine cause of fiery ...,"KENTON COUNTY, Ky. (WXIX) - The fiery explosio...","KENTON COUNTY, Ky. (WXIX) - The fiery explosio...",False,https://www.newsbreak.com/news/2483089711632/o...
3,3,2022-01-12,Cleves,OH,Injuries reported after crash on I-275 in Nort...,"FT. WRIGHT, Ky. (WKRC) - Injuries were reporte...","FT. WRIGHT, Ky. (WKRC) - Injuries were reporte...",False,https://www.newsbreak.com/news/2483307274479/i...
4,4,2022-01-11,Cleves,OH,Covington residents urged to shelter in place ...,"KENTON COUNTY, Ky. (WXIX) - A shelter-in-place...","KENTON COUNTY, Ky. (WXIX) - A shelter-in-place...",False,https://www.newsbreak.com/news/2482198518315/c...


In [12]:
all_df.shape

(26921, 9)

In [13]:
all_df = all_df.rename(columns={'Involves cyclist or pedestrian': 'cyclist_or_ped'})

Assemble a column with predictions regarding passive/active voice for each headline.

In [14]:
is_passive_column = []
for title in all_df["Title"].values:
    is_passive_column.append(int(is_passive(title)))
all_df["passive"] = is_passive_column

In [16]:
log_reg = smf.logit("passive ~ cyclist_or_ped", data=all_df).fit()

Optimization terminated successfully.
         Current function value: 0.237945
         Iterations 7


In [17]:
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:                passive   No. Observations:                26921
Model:                          Logit   Df Residuals:                    26919
Method:                           MLE   Df Model:                            1
Date:                Sun, 06 Feb 2022   Pseudo R-squ.:                 0.02550
Time:                        11:08:35   Log-Likelihood:                -6405.7
converged:                       True   LL-Null:                       -6573.3
Covariance Type:            nonrobust   LLR p-value:                 6.993e-75
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -2.7591      0.026   -105.169      0.000      -2.810      -2.708
cyclist_or_ped[T.True]     1.6333      0.078     20.808      0.000       1.479       1.787
